In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from webdriver_manager.chrome import ChromeDriverManager

import numpy as np
from numpy import random
from time import sleep

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.nytimes.com/games/wordle/index.html')

actions = ActionChains(driver)
actions.click()
actions.perform()

In [ ]:
wordleGuesses = np.loadtxt('wordle-guesses.txt', dtype = str)

In [ ]:
def makeRandomGuess():
    guessIdx = np.random.randint(low = 0, high = len(wordleGuesses))
    guess = wordleGuesses[guessIdx]
    actions.send_keys(guess)
    actions.send_keys(Keys.RETURN)
    actions.perform()

In [ ]:
for i in np.arange(6):
    makeRandomGuess()
    sleep(3)